В этом ноутбуке обучим модель MobileBERT распознавать спам в email

In [1]:
model_name = 'mobilebert'
train_dataset_name = 'spam emails'

In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [4]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [5]:
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.columns = ['DATA_COLUMN', 'IS_SPAM']

In [6]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


In [7]:
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))

In [8]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291 thi...,0
1,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs this deal is to b...,0
...,...,...
5166,Subject: put the 10 on the ft the transport vo...,0
5167,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,Subject: calpine daily gas nomination > > juli...,0
5169,Subject: industrial worksheets for august 2000...,0


In [9]:
df['IS_SPAM'].value_counts()

0    3672
1    1499
Name: IS_SPAM, dtype: int64

In [10]:
df_positive = df[df['IS_SPAM']==1]

In [11]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [12]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [13]:
df_negative_test.shape

(1836, 2)

In [14]:
df_positive_test.shape

(749, 2)

In [15]:
df_positive_test

,DATA_COLUMN,IS_SPAM
2699,Subject: your mother knows you watch girls moa...,1
2702,Subject: 65 % off for all new software . pyrrh...,1
2703,Subject: [ wrenches ] 68 % off dreamweaver mx ...,1
2706,Subject: he reached around and fingered me whi...,1
2709,"Subject: best meds around , paliourg haven ' t...",1
...,...,...
5159,Subject: pictures streamlined denizen ajar cha...,1
5161,Subject: penny stocks are about timing nomad i...,1
5162,Subject: anomaly boys from 3881 uosda apaprove...,1
5164,Subject: slutty milf wants to meet you take th...,1


In [16]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [17]:
df_balanced_test.sample(10)

,DATA_COLUMN,IS_SPAM
3154,"Subject: hpl nom for july 28 , 2000 ( see atta...",0
5113,"Subject: txu may 01 mike , i have the followin...",0
3113,Subject: hi there man - feel the vitality what...,1
4384,Subject: the only solution to penis enlargeme...,1
5134,Subject: private vl @ gra remove me,1
4563,Subject: want a new playstation 2 ?,1
5023,Subject: majestic rx - - - - 86738490004042205...,1
4488,Subject: never worry about money ever again $ ...,1
4902,"Subject: deal 107937 for 1 / 2000 daren , stac...",0
4503,Subject: tenaska iv outage update fyi . - - - ...,0


In [18]:
df_balanced_test['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [19]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [20]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [21]:
df_balanced_train['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [22]:
df_balanced_train.sample(10)

,DATA_COLUMN,IS_SPAM
941,Subject: we sell regalis for an affordable pri...,1
2355,Subject: cut 20 / d into midcon - - - - - - - ...,0
938,"Subject: conoco , inc . - missing deal for may...",0
522,Subject: haave yoou tried either of these pro ...,1
672,Subject: may nom daren the stuff outlined in b...,0
2090,"Subject: re : document samantha , please sched...",0
2332,Subject: via - ggra is lousy iimors ci - ialis...,1
541,Subject: ? ? ? ? ? ? ? ? erp ! ? ????? ? ? ? ...,1
2037,Subject: april nominations at shell deer park ...,0
1177,Subject: time may be running out homeowners - ...,1


In [23]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [24]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [25]:
mobilebert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") # same as bert


In [26]:
mobilebert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/mobilebert_en_uncased_L-24_H-128_B-512_A-4_F-4_OPT/1", trainable=True)

In [27]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = mobilebert_preprocess(text_input)
outputs = mobilebert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [28]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [29]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [31]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [32]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
81/81 [==============================] - 250s 473ms/step - loss: 54288.7422 - accuracy: 0.6062 - precision: 0.3209 - recall: 0.3218
Epoch 2/15
81/81 [==============================] - 35s 428ms/step - loss: 1.6222 - accuracy: 0.6484 - precision: 0.3895 - recall: 0.3765
Epoch 3/15
81/81 [==============================] - 37s 458ms/step - loss: 1.0504 - accuracy: 0.6882 - precision: 0.4601 - recall: 0.4393
Epoch 4/15
81/81 [==============================] - 34s 417ms/step - loss: 0.7502 - accuracy: 0.7346 - precision: 0.5457 - recall: 0.5020
Epoch 5/15
81/81 [==============================] - 31s 388ms/step - loss: 0.5473 - accuracy: 0.8066 - precision: 0.6751 - recall: 0.6409
Epoch 6/15
81/81 [==============================] - 30s 367ms/step - loss: 0.3691 - accuracy: 0.8615 - precision: 0.7689 - recall: 0.7463
Epoch 7/15
81/81 [==============================] - 31s 383ms/step - loss: 0.3162 - accuracy: 0.8925 - precision: 0.8248 - recall: 0.7984
Epoch 8/15
81/81 [===========

In [33]:
y_predicted = model.predict(X_test)
y_predicted


81/81 [==============================] - 15s 149ms/step


array([[1.0955226e-02],
       [2.2697130e-04],
       [3.8803792e-05],
       ...,
       [9.9999988e-01],
       [9.9999094e-01],
       [9.9968433e-01]], dtype=float32)

In [34]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [35]:
y_test

2536    0
2537    0
2539    0
2540    0
2542    0
       ..
5159    1
5161    1
5162    1
5164    1
5170    1
Name: IS_SPAM, Length: 2585, dtype: int64

In [36]:
accuracy_score(y_test, y_predicted)

0.9470019342359768

In [37]:
precision_score(y_test, y_predicted)


0.8713592233009708

In [38]:
recall_score(y_test, y_predicted)

0.9586114819759679

In [39]:
f1_score(y_test, y_predicted)

0.9129052765416401

In [40]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [41]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [42]:
df_results_on_test

,accuracy,precision,recall,f1_score
mobilebert trained on spam emails and tested on spam emails dataset,0.947002,0.871359,0.958611,0.912905


Сохраним обученную модель

In [43]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_emails_20_january'

In [44]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_emails_20_january


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [ ]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [ ]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [ ]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [54]:
df_with_train_quality

,accuracy,precision,recall,f1_score
mobilebert trained on spam emails quality on train dataset,0.984139,0.974531,0.970627,0.972575


In [ ]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [ ]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [ ]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')